In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
edges = pd.read_csv('edges_L6.csv', sep=',')
nodelist = pd.read_csv('nodes_l6.csv', sep=',', index_col=0)
G = nx.from_pandas_edgelist(edges, '# source', 'target', edge_attr = 'count')

In [ ]:
plant = nodelist[nodelist.pollinator == 0]
nx.set_node_attributes(G, dict(plant.pollinator), 'bipartite')
insect = nodelist[nodelist.pollinator == 1]
nx.set_node_attributes(G, dict(insect.pollinator), 'bipartite')

In [ ]:
bipartite.is_bipartite(G)

In [ ]:
node_set_insect = edges['# source']
node_set_plant = edges['target']

In [ ]:
projected_graph_insect = bipartite.weighted_projected_graph (G,node_set_insect, ratio = False)
projected_graph_plant = bipartite.weighted_projected_graph (G,node_set_plant, ratio = False)

In [ ]:
# Q1
print(f'Number of nodes in insect = {len(node_set_insect.drop_duplicates())}')
print(f'Number of nodes in plant = {len(node_set_plant.drop_duplicates())}')
print(f'Number of edges in the graph = {len(G.edges)}')

In [ ]:
# Q2
node_set_insect.value_counts().to_frame().head(5)
node_set_plant.value_counts().to_frame().head(5)

In [ ]:
# Q3
repeated_list = []
sorted_dict = {}

adj_A = nx.to_numpy_matrix(projected_graph_insect)
cos_sim_A = cosine_similarity(adj_A) # Create a cosine similarity matrix
idx = np.argsort(cos_sim_A, axis=None)[::-1] # Form an array to identify the sequence of the matrix value
rows, cols = np.unravel_index(idx, cos_sim_A.shape) # Form rows and cols array by identifying values' index
A_sorted = cos_sim_A[rows, cols] # Sort the value in orders to the matrix

for r, c, v in zip(rows, cols, A_sorted):
    total_no = r + c # Calculate the summation of the row and column number to avoid repeated value
    if (r == c or total_no in repeated_list): continue # Skip the loop if the value with the same row and column name
    else:
        sorted_dict[r, c] = v
        repeated_list.append(total_no) # Add the total number into the list for next loop checking

max_cos=dict(sorted(sorted_dict.items(), key=lambda x: x[1],reverse=True)[:5]) # Show all at once in descending order
print(max_cos)

In [ ]:
# Q4
nx.info(projected_graph_insect)
nx.info(projected_graph_plant)

In [ ]:
# Q5
degree_centrality_insect = nx.degree_centrality(projected_graph_insect)
max_key_insect= max(degree_centrality_insect, key=degree_centrality_insect.get)
max_value_insect = max(degree_centrality_insect.values())
print(max_key_insect, max_value_insect)

degree_centrality_plant = nx.degree_centrality(projected_graph_plant)
max_key_plant = max(degree_centrality_plant, key=degree_centrality_plant.get)
max_value_plant = max(degree_centrality_plant.values())
print(max_key_plant, max_value_plant)

In [ ]:
# Q6
list(nx.isolates(projected_graph_insect))
list(nx.isolates(projected_graph_plant))

In [ ]:
# Q7
def edge_to_remove(g): 
    d1 = nx.edge_betweenness_centrality(g) 
    list_of_tuples = list(d1.items()) 
      
    sorted(list_of_tuples, key = lambda x:x[1], reverse = True) 
    return list_of_tuples[0][0] 
  
def girvan(g): 
    a = nx.connected_components(g) 
    lena = len(list(a)) 
    print (' The number of connected components are ', lena) 
    while (lena == 1): 
        u, v = edge_to_remove(g) 
        g.remove_edge(u, v)  
          
        a = nx.connected_components(g) 
        lena=len(list(a)) 
        print (' The number of connected components are ', lena) 
    return a 

no_insect_cluster = girvan(projected_graph_insect)
no_plant_cluster = girvan(projected_graph_plant)

In [ ]:
# Q8
nx.degree_assortativity_coefficient(projected_graph_insect)
nx.degree_assortativity_coefficient(projected_graph_plant)

In [ ]:
# Q9
plt.figure(figsize=(16,36))
pos = nx.bipartite_layout(G, projected_graph_plant)
colors = ['red' if node[1]['bipartite']==0 else 'gray' for node in G.nodes(data = True)]
nx.draw_networkx(G, pos = pos, node_color = colors, width = 0.2, with_labels = False)

In [ ]:
plt.figure(figsize=(60,54))
degree_centrality_insect = nx.degree_centrality(projected_graph_insect)
nx.set_node_attributes(projected_graph_insect, degree_centrality_insect, 'degree_centrality')
sizes = [n[1]['degree_centrality'] * 3000 for n in projected_graph_insect.nodes(data = True)]
nx.draw_networkx(projected_graph_insect, node_size = sizes, with_labels = False, width = 0.2)

In [ ]:
plt.figure(figsize=(54,54))
degree_centrality_plant = nx.degree_centrality(projected_graph_plant)
nx.set_node_attributes(projected_graph_plant, degree_centrality_plant, 'degree_centrality')
sizes = [n[1]['degree_centrality'] * 1000 for n in projected_graph_plant.nodes(data = True)]
nx.draw_networkx(projected_graph_plant, node_size = sizes, with_labels = False, width = 0.2)